In [1]:
import sevenbridges as sbg
from sevenbridges.http.error_handlers import rate_limit_sleeper, maintenance_sleeper
import sys
import re
import pdb
import concurrent.futures
from requests import request
config = sbg.Config(profile='cavatica')
api = sbg.Api(config=config, error_handlers=[rate_limit_sleeper, maintenance_sleeper])

In [2]:
def cp_file(task):
    try:
        
        if re.search(prefix, task.name) and task.outputs[out_name].metadata[file_tag] in bs_dict and (suffix is None or re.search(suffix, task.name)):
            file_obj = task.outputs[out_name]
            new_cp = file_obj.copy(project=dest, name=file_obj.name)
            new_mv = new_cp.move_to_folder(parent=new_folder, name=new_cp.name)
            return new_mv.id + '\t' + new_mv.name + '\n'
    except Exception as e:
        sys.stderr.write(str(e) + '\n')
        sys.stderr.write('Could not process task ' + task.name + '\n')
        pass


In [4]:
# fixed info
dest = 'brownm28/cbttc-ipc'
maf = 'vep_annotated_maf'
rsem = 'RSEM_gene'
somatic_file_tag = 'Kids First Biospecimen ID Tumor'
tumor_only_file_tag = 'Kids First Biospecimen ID'
# likely to edit
prefix = 'RNAfusion-'
suffix = None # set to None if you don't need this
folder_name = 'rnaseq_expression'
src = 'kfdrc-harmonization/sd-bhjxbdqk-06'
out_name = rsem
file_tag = tumor_only_file_tag
metadata = open('/Users/brownm28/Documents/2018-Mar-28_cbttc_maf_req/cbttc_clinical_data.tsv')
out_fh = open('/Users/brownm28/Documents/2018-Mar-28_cbttc_maf_req/' + folder_name + '_copied_files.txt', 'a')

#set up
new_folder = ''
try:
    new_folder = api.files.create_folder(name=folder_name, project=dest)
except Exception as e:
    sys.stderr.write(str(e) + '\n')
    sys.stderr.write('Folder already exists, getting existing folder\n')
    new_folder = api.files.query(names=[folder_name], project=dest)[0]
head = next(metadata)
header = head.rstrip('\n').split('\t')
i = header.index('SPECIMEN_ID')
bs_dict = {}
for line in metadata:
    info = line.rstrip('\n').split('\t')
    ids = info[i].split(';')
    for bs_id in ids:
        bs_dict[bs_id] = 0
tasks = api.tasks.query(project=src, status='COMPLETED').all()

i = 1
n = 50
with concurrent.futures.ThreadPoolExecutor(16) as executor:
    results = {executor.submit(cp_file, task): task for task in tasks}
    for result in concurrent.futures.as_completed(results):
        if result.result() is not None:
            i += 1
            out_fh.write(result.result())
            if i % n == 0:
                sys.stderr.write(str(i) + ' files copied\n')
out_fh.close()

50 files copied
100 files copied
150 files copied
200 files copied
250 files copied
300 files copied
350 files copied
400 files copied
450 files copied
500 files copied
550 files copied
600 files copied
650 files copied
700 files copied
750 files copied
800 files copied
850 files copied
900 files copied
950 files copied
1000 files copied
